**TLDR**: <ins>WHY</ins> do tokenizers represent spaces `" "` with the Unicode character `Ġ`?

Hello, World!

While working on a different blog post where I plan to dive deeper into training LLM tokenizers using the Hugging Face `tokenizers` library, I could not help but notice the `Ġ` Unicode character being omnipresent in the vocabularies of seemingly **all** tokenizers based on the <a href="https://en.wikipedia.org/wiki/Byte_pair_encoding" target="_blank">*byte pair encoding* (BPE)</a> algorithm. I also noticed that it has something to do with replacing the space `" "` character, for example, a `Llama 3 8B Instruct` tokenizer would produce the following outputs:

```python
>>> tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
>>> tokenizer.tokenize("Some text to be tokenized")
['Some', 'Ġtext', 'Ġto', 'Ġbe', 'Ġtoken', 'ized']
```

I have tried looking for an answer online, but found out that people were as confused as I was, below are a few issues on GitHub only:

1. <a href="https://github.com/openai/gpt-2/issues/80" target="_blank">Why \u0120 (Ġ) is in so many pairs?</a>
1. <a href="https://github.com/openai/gpt-2/issues/185" target="_blank">Confused about vocab.bpe and encoder.json</a>
1. <a href="https://github.com/allenai/OLMo/issues/505" target="_blank">Could some one help why the tokens begin with G.</a>

There are many other similar questions, but all of the answers seem to simply suggest that <a href="https://github.com/huggingface/transformers/issues/3867#issuecomment-616956437)" target="_blank">it is a feature of the BPE algorithm</a>, but neither of the discussions answered **why** should such a feature exists in the first place.

This blog post is my attempt at finding a more satisfying answer which (spoiler alert!) seems to be that replacing spaces with the character `Ġ` could well be a historical artifact and not at all a strict design requirement. Read on to see what I came up with!

# Approach

After doing some background reading, I found that it is essentially the <a href="https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf" target="_blank">`GPT-2`</a> paper that popularized the use of BPE-based tokenizers. Digging through <a href="https://github.com/openai/gpt-2/" target="_blank">their code</a>, I came up with the following idea: <ins>*if there was never a good reason to replace spaces with the character `Ġ`, could we revert back to using spaces for encoding and decoding strings?*</ins>

More specifically, we are going to do the following:

1. Refactor OpenAI's code of the original `GPT-2` tokenizer to be able to operate spaces instead of `Ġ's`.
1. Test on a diverse dataset to see if the encoding behaviour remains the same.
    - It turns out that we will not need to change the decoding functionality, so long as encoding behaves the same, we do not need to test the decoding bit.

Let's jump into the code!

# Refactoring the GPT-2 tokenizer code

First, let us see how the original `GPT-2` tokenizer's code has to be changed to accommodate spaces. The original implementation can be found <a href="https://github.com/openai/gpt-2/blob/master/src/encoder.py" target="_blank">here</a>. The collapsed cell below downloads the files necessary for the `GPT-2` tokenizer to work. 

:::{.callout-note}
Somewhat confusingly, the original authors of the repository named the files `vocab.bpe` and `encoder.json`, whereas more appropriate names would be `marges.bpe` and `vocab.json` based on the contents of the files.
:::

In [2]:
#|code-fold: true
#|output: false
%load_ext autoreload
%autoreload 2

In [3]:
#|code-fold: true
#|output: false
from pathlib import Path
from tqdm.notebook import tqdm


Path("gpt2-tokenizer").mkdir(exist_ok=True)

GPT2_URL_BASE = "https://openaipublic.blob.core.windows.net/gpt-2/models/1558M"
if not Path("gpt2-tokenizer/vocab.bpe").exists():
    !wget -q {GPT2_URL_BASE}/vocab.bpe -P gpt2-tokenizer/
if not Path("gpt2-tokenizer/encoder.json").exists():
    !wget -q {GPT2_URL_BASE}/encoder.json -P gpt2-tokenizer/


<font color="red">The first change that we have to make is...</font>

<font color="red">Explain what the `get_bytes_to_unicode_base_mapping()` is and why it exists, highlight that it is where the `Ġ's` would be introduced, but emphasize that it does not seem like spaces would not work</font>

In [4]:
from gpt2_tokenizer_custom import get_bytes_to_unicode_base_mapping

list(get_bytes_to_unicode_base_mapping().items())[:5]

[(33, '!'), (34, '"'), (35, '#'), (36, '$'), (37, '%')]

In [13]:
list(tokenizer_original.encoder.items())[:256]

[('!', 0),
 ('"', 1),
 ('#', 2),
 ('$', 3),
 ('%', 4),
 ('&', 5),
 ("'", 6),
 ('(', 7),
 (')', 8),
 ('*', 9),
 ('+', 10),
 (',', 11),
 ('-', 12),
 ('.', 13),
 ('/', 14),
 ('0', 15),
 ('1', 16),
 ('2', 17),
 ('3', 18),
 ('4', 19),
 ('5', 20),
 ('6', 21),
 ('7', 22),
 ('8', 23),
 ('9', 24),
 (':', 25),
 (';', 26),
 ('<', 27),
 ('=', 28),
 ('>', 29),
 ('?', 30),
 ('@', 31),
 ('A', 32),
 ('B', 33),
 ('C', 34),
 ('D', 35),
 ('E', 36),
 ('F', 37),
 ('G', 38),
 ('H', 39),
 ('I', 40),
 ('J', 41),
 ('K', 42),
 ('L', 43),
 ('M', 44),
 ('N', 45),
 ('O', 46),
 ('P', 47),
 ('Q', 48),
 ('R', 49),
 ('S', 50),
 ('T', 51),
 ('U', 52),
 ('V', 53),
 ('W', 54),
 ('X', 55),
 ('Y', 56),
 ('Z', 57),
 ('[', 58),
 ('\\', 59),
 (']', 60),
 ('^', 61),
 ('_', 62),
 ('`', 63),
 ('a', 64),
 ('b', 65),
 ('c', 66),
 ('d', 67),
 ('e', 68),
 ('f', 69),
 ('g', 70),
 ('h', 71),
 ('i', 72),
 ('j', 73),
 ('k', 74),
 ('l', 75),
 ('m', 76),
 ('n', 77),
 ('o', 78),
 ('p', 79),
 ('q', 80),
 ('r', 81),
 ('s', 82),
 ('t', 83),
 

<font color="red">Other code changes are ...</font>

```python
def bytes_to_unicode():
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8+n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))

# --- vs ---

def get_bytes_to_unicode_base_mapping():
    # Return almost exactly the same mapping, just with the space character "Ġ" replaced with " "
    return {**bytes_to_unicode(), 32: " "}
```

```python
class GPT2TokenizerCustom(Encoder):
    def __init__(self, encoder, bpe_merges, errors="replace"):
        # Only the byte_encoder attribute is changed, everything else is the same
        super().__init__(encoder, bpe_merges, errors)
        self.byte_encoder = get_bytes_to_unicode_base_mapping()

    def bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        word = tuple(token)
        pairs = get_pairs(word)

        if not pairs:
            return token

        while True:
            bigram = min(pairs, key=lambda pair: self.bpe_ranks.get(pair, float("inf")))
            if bigram not in self.bpe_ranks:
                break
            
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
                    new_word.append(first + second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        
        # --- New code ---
        
        # Old code: `word =  " ".join(word)`

        # New code:
        word = tuple(word)

        # --- End of new code ---
        
        self.cache[token] = word
        
        return word

    def encode(self, text):
        bpe_tokens = []
        for token in re.findall(self.pat, text):
            token = "".join(self.byte_encoder[b] for b in token.encode("utf-8"))
            
            # --- New code ---
            
            # Old code:
            # `bpe_tokens.extend(self.encoder[bpe_token] for bpe_token in self.bpe(token).split(' '))`
            
            # New code: the split(" ") is removed:
            bpe_tokens.extend(self.encoder[bpe_token] for bpe_token in self.bpe(token))
            
            # --- End of new code ---
        
        return bpe_tokens

```

# Test the refactored code

In [6]:
from gpt2_tokenizer import get_encoder
from gpt2_tokenizer_custom import get_encoder_custom

tokenizer_original = get_encoder("", "gpt2-tokenizer")
tokenizer_custom = get_encoder_custom("", "gpt2-tokenizer")

simple_test_string = "ĠHello've world123 how's     it going!!!?   \u0120 anotherĠ"

In [7]:
expected_output = tokenizer_original.encode(simple_test_string)
print(", ".join(map(str, expected_output)))

output = tokenizer_custom.encode(simple_test_string)
print(", ".join(map(str, output)))

assert output == expected_output

128, 254, 15496, 1053, 995, 10163, 703, 338, 220, 220, 220, 220, 340, 1016, 10185, 30, 220, 220, 34754, 254, 1194, 128, 254
128, 254, 15496, 1053, 995, 10163, 703, 338, 220, 220, 220, 220, 340, 1016, 10185, 30, 220, 220, 34754, 254, 1194, 128, 254


## Test on a more diverse dataset

In [8]:
from datasets import load_dataset

split = "train"
english_dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split=split)
korean_dataset = load_dataset("lcw99/wikipedia-korean-20221001", split=split)
code_dataset = load_dataset("code_search_net", "python", split=split, trust_remote_code=True)
code_dataset = code_dataset.rename_column("whole_func_string", "text")  # Rename whole_func_string to text

n = 10000
final_dataset = (
    english_dataset.shuffle(42).select(range(n))["text"] +
    korean_dataset.shuffle(42).select(range(n))["text"] +
    code_dataset.shuffle(42).select(range(n))["text"]
)
print(f"{len(final_dataset)=}")

len(final_dataset)=30000


Let's make sure that the Korean data is in there somewhere:

In [9]:
final_dataset[10001][:50]

'김혜리(1969년 12월 23일 ~ )는 대한민국의 배우이며, 1988년 미스코리아 선 출'

Now run the tests!

In [10]:
for idx, test_string in tqdm(enumerate(final_dataset), total=len(final_dataset)):
    expected_output = tokenizer_original.encode(test_string)
    output = tokenizer_custom.encode(test_string)

    if output != expected_output:
        print(f"{idx=}")


  0%|          | 0/30000 [00:00<?, ?it/s]

# Conclusion

The tests above suggest that:

1. It might be an artifact:
    - Could it be a bug?
    - Maybe our tests are not extensive enough?
1. Is there something that we do not know?
    - We might never know...

Result: much more satisfying, but still inconclusive

# Sources

1. Let's build the GPT Tokenizer (<a href="https://www.youtube.com/watch?v=zduSFxRajkE" target="_blank">video</a>) by Andrej Karpathy.
1. gpt-2 (<a href="https://github.com/openai/gpt-2/" target="_blank">GitHub repository</a>)
1. https://discuss.huggingface.co/t/how-to-add-additional-custom-pre-tokenization-processing/1637
1. https://huggingface.co/learn/nlp-course/en/chapter6/5#implementing-bpe
1. <a href="https://github.com/huggingface/tokenizers/blob/71c2a8d01a56cd7bd28148c309e210c47dac78e7/bindings/python/py_src/tokenizers/__init__.pyi#L420" target="_blank">`NormalizedString`</a>
1. <a href="https://github.com/huggingface/tokenizers/blob/b24a2fc1781d5da4e6ebcd3ecb5b91edffc0a05f/bindings/python/examples/custom_components.py" target="_blank">Custom HF components</a>
